In [16]:
! jupyter nbconvert --to python Grid_World_Environment.ipynb


[NbConvertApp] Converting notebook Grid_World_Environment.ipynb to python
[NbConvertApp] Writing 3086 bytes to Grid_World_Environment.py


In [17]:
import numpy as np
from Grid_World_Environment import standard_grid, ACTION_SPACE

In [18]:
SMALL_ENOUGH = 1e-3 # threshold for convergence
gamma = 0.9 # discount factor

In [19]:
def print_values(V, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")


def print_policy(P, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")


In [20]:
transition_probs = {}

  # to reduce the dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  # note: you could make it simpler by using r(s') since the reward doesn't
  # actually depend on (s, a)
rewards = {}

grid = standard_grid()
for i in range(grid.rows):
    for j in range(grid.cols):
      s = (i, j)
      if not grid.is_terminal(s):
        for a in ACTION_SPACE:
          s2 = grid.get_next_state(s, a)
          transition_probs[(s, a, s2)] = 1
          if s2 in grid.rewards:
            rewards[(s, a, s2)] = grid.rewards[s2]


In [21]:
  ### fixed policy ###
policy = {
    (2, 0): 'D',
    (1, 0): 'D',
    (0, 0): 'D',
    (0, 1): 'D',
    (0, 2): 'D',
    (1, 2): 'D',
    (2, 1): 'D',
    (2, 2): 'R',
    (2, 3): 'D',
  }
print_policy(policy, grid)

---------------------------
  D  |  D  |  D  |     |
---------------------------
  D  |     |  D  |     |
---------------------------
  D  |  D  |  R  |  D  |


In [22]:
# initialize V(s) = 0
V = {}
for s in grid.all_states():
  V[s] = 0

In [23]:
# repeat until convergence
it = 0
while True:
    biggest_change = 0
    for s in grid.all_states():
        if not grid.is_terminal(s):
            old_v = V[s]
            new_v = 0 # we will accumulate the answer
            for a in ACTION_SPACE:
                for s2 in grid.all_states():
                    # action probability is deterministic
                    action_prob = 1 if policy.get(s) == a else 0
                    
                    # reward is a function of (s, a, s'), 0 if not specified
                    r = rewards.get((s, a, s2), 0)
                    new_v += action_prob * transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

            # after done getting the new value, update the value table
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    print("*********************************************")
    print("iter:", it, "biggest_change:", biggest_change)
    print_values(V, grid)
    it += 1

    if biggest_change < SMALL_ENOUGH:
        break
print("\n\n")


*********************************************
iter: 0 biggest_change: 0
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|



